# Zajęcia 1 (część 1)

Zakres:
* Zapoznanie się ze srodowiskiem pracy (Dataproc na Google Cloud Platform)
* Operacje na obiektowym systemie plików (GCS)
* Interaktywna analiza rozproszonych danych przy wykorzystaniu modułu Spark Core i Spark SQL
* Wprowadzenie modułu pandas
* Wizualizacje przy wykorzystaniu matplotlib oraz seaborn

Języki:
* bash, Python, SQL 

Krótka lista przydatnych poleceń bash: https://www.reddit.com/r/linux/comments/9rns12/some_linux_commands_cheatsheet/

### Jupyter Notebook

* Interaktywny notatnik dostepny poprzez przeglądarkę.
* Służy do wpisywania poleceń w wybranych jezykach programowania oraz opis w tzw jezyku markdown.
* Polecenia/opis wpisujemy do komórek. Komórki są wykonywane przez tzw. kernel (np Python w określonej wersji)
* Działa w tryb edycji komórek + tryb wykonywania poleceń. (przejście poprzez ESC i ENTER)
* Podstawowe skróty klawiaturowe: 
  * Esc/Enter
  * strzalki
  * ctrl-enter/shift-enter
  * a/b/d
  * y/m 
* Moze wykonywać polecenia powłoki (bash) !  %%bash
* Notatnik zapisywany jest w formacie ipynb (IPythonNoteBook)
* Kolejnosc uruchomienia komórek moze byc rozna.
* Mozliwosc zatrzymania kernela i uruchomienia od nowa. 

Polecamy film: https://www.youtube.com/watch?v=HW29067qVWk

Przykładowe komórki Pythonowe i Bashowe:

In [ ]:
import os
user = os.environ.get('USER')
msg = f"Hello {user}!"
print (msg)

In [ ]:
! pwd # pokaż aktualną scieżkę

In [ ]:
%%bash
cd /root/ds-notebooks/session_1/
head 01_intro.ipynb
echo "--------------------"
tail 01_intro.ipynb

### Kopiowanie danych do swojego katalogu domowego

W katalogu `session1` znajduje sie plik który bedzie nam dzisiaj słuzyl do pracy.  
Dane pochodzą z https://gdac.broadinstitute.org/ i zawierają dane z badaniami nad mutacjami typu CNV w obrebie genu BRCA2.


In [ ]:
%%bash
cd /root/ds-notebooks/session_1/
mkdir -p ~/data     # stworz katalog data (i posrednie)
cp brca.txt ~/data  # skopiuj plik z lokalizacji do bieżącej lokalizacji
ls -lah   ~/data       # wylistuj zawartość bieżącego katalogu

In [ ]:
%%bash
head ~/data/brca.txt   # pokaż początek pliku
echo                   # pusta linia
tail ~/data/brca.txt   # pokaż koniec pliku 
echo
wc -l ~/data/brca.txt  # policz linie pliku

## Środowisko Google Cloud Dataproc

### TODO Dataproc intro



## Wykorzystanie Google Cloud Storage

#### Projekt
* Wszystkie dane przynależą do konkretnego projektu.
* Do projektu mogą mieć dostęp użytkownicy. 
* Projekt ma zdefiniowane metody uwierzytelniające, rozliczenia, monitorowanie etc. 
#### Kubełek (bucket)
* Kubełek (bucket) to kontener na pliki/obiekty. 
* Nazwa Bucketu musi być unikalna w skali całej usługi u wszystkich użytkowników (!) 
* Kubełków nie można zagnieżdzać
* W kubełkach możemy tworzyć foldery i tam logicznie grupować pliki.
* Kubełek wraz z zawartością może zostać udostępniony publicznie.
* Kubełkowi nie można zmienić nazwy lub metadanych. Trzeba go usunąć i stworzyć ponownie.
#### Obiekt 
* obiekty przechowywane w kubełkach 
* obiekty mają zawartość oraz metadane
* obiekty są niemodyfikowalne 

Do operacji na Google Storage można wykorzystać narzędzie `gsutil`: 

##### Operacje na kubełkach
* listowanie kubełków (buckets) - `ls`
* tworzenie nowego kubełka - `mb`
* usuwania kubełka - `rm`
* listowanie zawartości kubełków - `ls` 
* udostępnianie - `iam`

##### Operacje na obiektach
* dodawania pliku do kubełka - `cp`
* kopiowanie - `cp`
* usuniecie z kubełka - `cp`
* pobranie informacji o obiekcie - `stat`

#### Operacje na kubełkach

In [ ]:
! gsutil ls

In [ ]:
! echo $USER  # wyswietlenie nazwy uzytkownika

In [ ]:
# TODO Please update user id
%env USER_ID=9903
%env SEMESTER=2024l

In [ ]:
! gsutil mb gs://big-data-lab-$SEMESTER-$USER_ID

In [ ]:
! gsutil ls gs://big-data-lab-$SEMESTER-$USER_ID

In [ ]:
! gsutil du -s gs://big-data-lab-$SEMESTER-$USER_ID # ile zajmuje przestrzeni?

In [ ]:
! gsutil du -L -b gs://big-data-lab-$SEMESTER-$USER_ID # listowanie zawartości 

#### Operacje na zawartości kubełków

In [ ]:
! gsutil ls -r gs://big-data-lab-$SEMESTER-$USER_ID  # listowanie zawartosci kubełka

In [ ]:
! gsutil cp ~/ds-notebooks/README.md gs://big-data-lab-$SEMESTER-$USER_ID  # upload obiektu do kubełka

In [ ]:
! gsutil ls -r gs://big-data-lab-$SEMESTER-$USER_ID  # listowanie zawartosci kubełka

In [ ]:
! gsutil iam get gs://big-data-lab-$SEMESTER-$USER_ID

In [ ]:
! gsutil stat gs://big-data-lab-$SEMESTER-$USER_ID/README.md # metadane obiektu w kubełku

#### Przeniesienie do docelowego kubełka

## <span style='background:yellow'> ZADANIE 1 </span>
Wgraj plik brca.txt do kubełka `gs://ds-$SEMESTER-$USER_ID-notebook-data` do katalogu `data`.

## <span style='background:yellow'> ZADANIE 2 </span>
a) Utwórz plik zawierający aktualną datę i godzinę.  
b) Wrzuć plik do kubełka `gs://big-data-lab-$SEMESTER-$USER_ID`.

### Pobranie pliku z powrotem na lokalny dysk

In [ ]:
! gsutil cp gs://ds-$SEMESTER-$USER_ID-notebook-data/data/brca.txt .  # gdzie sie zapisal ten plik?

## Apache Spark

* Apache Spark - platforma ogólnego zastosowania, open source, do przetwarzania duzych zbiorów danych.
* Posiada API dla języków programowania: Scala, Python i R. 
* Przetwarzanie w Spark przetwarzanie jest wykonywane w większości  wprost w pamięci operacyjnej.
* Przeznaczenie: do uruchamiania  aplikacji i skryptów z wykorzystaniem uczenia maszynowego lub interaktywnych kwerend.
* Spark ten wspiera SQL (typ DataFrames), przetwarzanie strumieniowe oraz przetwarzanie grafów.
* Integracja z lokalną pamięci masową, rozproszonymi lub obiektowymi systemu plików.
* Spark można uruchamić na pojedynczej maszynie na środowisku klastrowym, lub w chmurze. 

Zakres na dzisiejsze laboratorium:
* stworzenie sesji Spark
* zaczytanie danych z pliku tekstowego
* kwerendy na danych

Będziemy korzystać z pliku ktory zapisalismy na GCS w pierwszej części ćwiczenia.

In [ ]:
! gsutil ls -r gs://ds-$SEMESTER-$USER_ID-notebook-data

### Przygotowanie sesji Sparkowej


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.config("spark.executor.instances", "1") \
.config("spark.executor.memory", "1g") \
.getOrCreate()

In [ ]:
spark.stop()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.config("spark.executor.instances", "1") \
.config("spark.executor.memory", "1g") \
.getOrCreate()

### Ręczne utworzenie zbioru danych

In [ ]:
# stworzenie rozproszonego DF na podstawie podanej listy krotek 
df_users = spark.createDataFrame(
    [(123, 'Jan Kowalski', 'jan@kowalski.pl'), 
     (124, 'Anna Nowak', 'anna@nowak.pl'), 
     (125, 'Janusz Kowalski', 'janusz@kowalski.pl'), 
     (126, 'Anita Nowak', 'anita@nowak.pl')])

In [ ]:
df_users.show()

In [ ]:
df_users.printSchema()

In [ ]:
df_users = spark.createDataFrame(
    [(123, 'Jan Kowalski', 'jan@kowalski.pl'), 
     (124, 'Anna Nowak', 'anna@nowak.pl'), 
     (125, 'Janusz Kowalski', 'janusz@kowalski.pl'), 
     (126, 'Anita Nowak', 'anita@nowak.pl')],
    schema='id int not null, name string, email string')

In [ ]:
df_users.show()

In [ ]:
df_users.printSchema()


### Wczytanie danych
Zazwyczaj dane mamy przechowane na zewnątrz naszego notatnika (np. w pliku) i należy je odczytać.

In [ ]:
# TODO Please update user id
semester = '2024l'
user_id = '9903'
input_path = f'gs://ds-{semester}-{user_id}-notebook-data/data/brca.txt'

In [ ]:
df = spark.read.load(input_path, format="csv", sep="\t", inferSchema="true", header="true")

### Charakterystyka danych

In [ ]:
type (df)                 # jaki jest typ danych

In [ ]:
df.explain()              # fizyczny plan wykonania

In [ ]:
df.explain(True)          # logiczny i fizyczny plan wykonania

In [ ]:
df.rdd.getNumPartitions() # liczba partycji (bloków danych)

In [ ]:
df.printSchema()          # schemat danych

In [ ]:
df.count()                # wymiary (liczba wierszy)

In [ ]:
len(df.columns)           # wymiary (liczba kolumn)

In [ ]:
df.describe().show()      # pokaż podsumowanie danych w tabeli

In [ ]:
df.describe("Chromosome").show() # pokaż podsumowanie danych w kolumnie 

### Odczyt danych
Z wykorzystaniem API DataFrame

In [ ]:
df.show()                    # pokaż 20 wierszy danych 

In [ ]:
df.show(10, truncate=False) # pokaż 10 wierszy, nie skracaj danych

In [ ]:
df.select("Sample").show()  # pokaż tylko kolumne sample

In [ ]:
df.select("Chromosome", "Start", "End").show() # pokaż kolumny chromosome, start i end

In [ ]:
df_chrom = df.select(df.Chromosome).distinct() # stworz nowy DF zawierajacy tylko unikalne wartosci chromosomow

In [ ]:
df_chrom.count()                     # policz wiersze w nowym DF

In [ ]:
df.filter("Chromosome > 21").show() # pokaz dane spelniajace warunek (przy uzyciu filter)

In [ ]:
df.where("Chromosome > 21").show() # pokaz dane spelniajace warunek (przy uzyciu when)

In [ ]:
df.filter("Chromosome > 21").explain()  # pokaz plan wykonania

In [ ]:
df.filter("Chromosome > 21 and Segment_Mean > 0").show() # pokaz dane spelniajace warunki

### Grupowanie i funkcje agregujące

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df.groupBy("Chromosome").count().show()   # dokonaj grupowania po chromosomie i policz rekordy w grupie

In [ ]:
df.groupBy("Chromosome").avg("Segment_Mean").show() # dokonaj grupowania po chromosomie i policz srednia wartosc segment_mean w grupie

In [ ]:
# pogrupuj dane wzgledem probki i chromosomu, policz rekordy w grupie
df.groupBy("Sample","Chromosome").count().orderBy(asc("Sample")).show()

### Kolumny wyliczane

In [ ]:
df.withColumn ("Length", col("End") - col("Start")).show() # dodaj kolumne dlugosc jako end - start

In [ ]:
df.withColumn ("Material", lit("DNA")).show()     # dodaj kolumne o stalej wartosci 'DNA'

In [ ]:
df.withColumn ("Chromosome2", concat(lit("chr"), col("Chromosome"))).show() # dodaj kolumne z konkatencja

In [ ]:
df.drop("Chromosome").show()  # usun kolumne Chromosome. 
# Czy DF została pozbawiona kolumny na trwale?

### Zapis wyników

In [ ]:
df2 = df.withColumn ("Chromosome2", concat(lit("chr"), col("Chromosome")))  # dodaj kolumne z konkatenacja i zapisz do nowego DF

In [ ]:
output_path = f'gs://ds-{semester}-{user_id}-notebook-data/data/brca2.txt'
df2.write.format("csv").mode("overwrite").save(output_path)

In [ ]:
! gsutil ls gs://ds-$SEMESTER-$USER_ID-notebook-data/data/

In [ ]:
!gsutil cat gs://ds-$SEMESTER-$USER_ID-notebook-data/data/brca.txt | head

#### Wazne!
Na koniec pracy przy tym notatniku (po wykonaniu poniższego polecenia) koniecznie zatrzymaj sesję Sparkową:

In [ ]:
spark.stop()

## <span style='background:yellow'> ZADANIE 3</span>

a) Ile jest unikalnych danych próbek w tym zbiorze danych?

b) Pokaż nazwę próbki, numer chromosomu, początek i koniec segmentu dla segmentów występujących na chromosomie 21,22,23 i mających startową pozycję większą niż 10000000. 

c) Ile występuje wierszy dla każdej z próbki? Pokaż wynik z pełną nazwą próbki. Zapisz wynik do pliku na GCS